# Non-Negativ Matrix Factorization Model on 1-gram Tokens

In [1]:
import warnings
warnings.simplefilter('ignore')
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pyLDAvis
import pyLDAvis.sklearn
from src.models.build_topic_model import show_topics

C:\Users\poorj\Projects\CCC-Topic-Modeling\env\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Load dataset

In [2]:
path_to_data = Path('../data')
df = pd.read_csv(path_to_data / 'processed' /'tokenized1gram_data.csv', converters={'tokenized': eval, 'tokenized_mults': eval, 'tokenized_mults_extr': eval})

In [14]:
df.head()

,file_name,text,word_count,tokenized_raw,tokenized_raw_cnt,tokenized_mults,tokenized_mults_cnt,tokenized_mults_extr,tokenized_mults_extr_cnt
0,07211613C1,Üdvözlöm Ha rádióra vagyok miben segíthetek le...,275,"['üdvözlö', 'rádió', 'miben', 'segíthet', 'les...",143,"[üdvözlö, rádió, miben, segíthet, lesz, csókol...",133,"[rádió, helyzet, elcsúszik, kötelező, gépjármű...",98
1,07313221C2,Üdvözlöm miről Barbara vagyok miben segíthetek...,197,"['üdvözlö', 'mi', 'miben', 'segíthet', 'ön', '...",90,"[üdvözlö, mi, miben, segíthet, ön, nap, kíván,...",86,"[nap, probléma, lejár, úgymond, kötelező, éves...",60
2,07372317C1,esti Papp Ildikó biztosítótól és ki tudunk bes...,385,"['esti', 'biztosító', 'tud', 'beszél', 'igazáb...",171,"[esti, biztosító, tud, beszél, igazából, szüle...",164,"[esti, biztosító, tud, beszél, szüleim, lakás,...",127
3,07452941C2,Üdvözlöm Lőrincz Barbara vagyok miben segíthet...,448,"['üdvözlö', 'miben', 'segíthet', 'ön', 'nap', ...",216,"[üdvözlö, miben, segíthet, ön, nap, eszik, elő...",204,"[nap, eszik, beszél, ügyintéző, tud, jármű, bi...",155
4,07504737C1,Üdvözlöm Horváthné Papp Linda vagyok miben seg...,671,"['üdvözlöm', 'miben', 'segíthet', 'reggel', 'k...",299,"[üdvözlöm, miben, segíthet, reggel, kíván, seg...",288,"[reggel, segítség, szükség, gépjármű, biztosít...",223


### Fit model for full token set (incl. extra stops)

In [3]:
tfidf = TfidfVectorizer(analyzer='word', tokenizer=lambda doc: doc, preprocessor = lambda doc: doc, token_pattern=None)
dtm_tfidf = tfidf.fit_transform(df['tokenized_mults'])
vocab = np.array(tfidf.get_feature_names())

nmf_tfidf = NMF(init='random', n_components=6, max_iter=100, random_state = 0)
nmf_tfidf.fit(dtm_tfidf)

pyLDAvis.enable_notebook()
vis_nmf = pyLDAvis.sklearn.prepare(nmf_tfidf, dtm_tfidf, tfidf)
vis_nmf

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.076174  0.027879       1        1  22.047664
2     -0.038425  0.063448       2        1  19.961473
1      0.083982 -0.138478       3        1  16.997011
0      0.007428 -0.126556       4        1  14.847283
5     -0.262575  0.025989       5        1  13.833141
4      0.133417  0.147718       6        1  12.313428, topic_info=            Term       Freq      Total Category  logprob  loglift
666        csekk  45.000000  45.000000  Default  30.0000  30.0000
1911  hozzájárul  26.000000  26.000000  Default  29.0000  29.0000
350         bank  21.000000  21.000000  Default  28.0000  28.0000
1976       hónap  31.000000  31.000000  Default  27.0000  27.0000
101      ajánlat  22.000000  22.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
818          díj   5.471199  34.887808   Topic6  -5.3577   0.2418
859          egy   4.146780  15.909119   Topic6  -5.6349   0.7499
1868      hogyha   4.361398  24.994683   Topic6  -5.5844   0.3486
3422        mond   4.584513  37.680117   Topic6  -5.5345  -0.0120
47            ad   4.421930  32.039762   Topic6  -5.5707   0.1141

[541 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
2         2  1.321172             157es
47        1  0.124845                ad
47        2  0.218479                ad
47        3  0.187267                ad
47        4  0.093634                ad
...     ...       ...               ...
5486      1  0.922071            ügyfél
5486      3  0.061471            ügyfél
5490      5  1.039443      ügyfélprofil
5492      2  0.683806  ügyfélszolgálati
5498      5  1.000605            ügynök

[740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1, 6, 5])

In [4]:
W1 = nmf_tfidf.fit_transform(dtm_tfidf)
H1 = nmf_tfidf.components_

show_topics(H1, num_top_words = 8, vocab = vocab)

['díj ugye évforduló biztosítás éves ft szerződés fizet',
 'csekk küld fizet tud befizet kap hónap hó',
 'tud cím email jelszó küld felület ön online',
 'tud szia utal mond szerződés ügyfél összeg lát',
 'bank hónap csoportos megbízás levon tud von terhelés',
 'hozzájárul ajánlat adat köszön elérhetőség szerződés megkeres akció']

### Fit model for cleaned token set (excl. extra stops)

In [21]:
tfidf = TfidfVectorizer(analyzer='word', tokenizer=lambda doc: doc, preprocessor = lambda doc: doc, token_pattern=None)
dtm_tfidf = tfidf.fit_transform(df['tokenized_mults_extr'])
vocab = np.array(tfidf.get_feature_names())

nmf_tfidf = NMF(init='random', n_components=8, max_iter=100, random_state = 0)
nmf_tfidf.fit(dtm_tfidf)

pyLDAvis.enable_notebook()
vis_nmf = pyLDAvis.sklearn.prepare(nmf_tfidf, dtm_tfidf, tfidf)
vis_nmf

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.058999 -0.018906       1        1  19.432923
2     -0.036668  0.106099       2        1  14.476090
6     -0.099996 -0.106536       3        1  14.129979
1      0.112990 -0.100502       4        1  12.041235
0      0.003052  0.069950       5        1  11.859902
7     -0.125798  0.177091       6        1  10.146053
4      0.280387  0.045043       7        1   9.075446
3     -0.074968 -0.172239       8        1   8.838371, topic_info=            Term       Freq      Total Category  logprob  loglift
654        csekk  40.000000  40.000000  Default  30.0000  30.0000
1877  hozzájárul  20.000000  20.000000  Default  29.0000  29.0000
339         bank  20.000000  20.000000  Default  28.0000  28.0000
2151      jelszó  21.000000  21.000000  Default  27.0000  27.0000
688    csoportos  16.000000  16.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
526    biztosító   2.747850  12.346575   Topic8  -5.5992   0.9235
47            ad   3.292938  33.109200   Topic8  -5.4182   0.1180
4710        utal   3.228692  30.061755   Topic8  -5.4380   0.1949
317    azonosító   2.964425  21.759483   Topic8  -5.5233   0.4327
1937          hó   2.767255  29.622962   Topic8  -5.5922   0.0554

[617 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2         4  1.372047            157es
47        1  0.302031               ad
47        2  0.090609               ad
47        3  0.060406               ad
47        4  0.090609               ad
...     ...       ...              ...
5363      7  0.974143     ügyfélprofil
5364      2  0.310801  ügyfélszolgálat
5364      4  0.621601  ügyfélszolgálat
5364      5  0.155400  ügyfélszolgálat
5371      7  0.974710           ügynök

[954 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 7, 2, 1, 8, 5, 4])

In [22]:
W1 = nmf_tfidf.fit_transform(dtm_tfidf)
H1 = nmf_tfidf.components_

show_topics(H1, num_top_words = 8, vocab = vocab)

['levél kap biztosítás kötelező casco küld gépjármű nap',
 'jelszó tud cím email felület online felhasználónév regisztráció',
 'díj évforduló éves fizet szerződés hó következő biztosítás',
 'bank csoportos megbízás levon von beszedési levonás terhelés',
 'hozzájárul ajánlat adat elérhetőség akció megkeres szerződés álló',
 'tud szerződés név információ ért telefon meghatalmazás szerződő',
 'utal összeg közlemény számlaszám lát ír utalás pénz',
 'csekk küld befizet fizet kap hó negyedév darab']

In [65]:
print(W1.shape, H1.shape, vocab.shape)

(1000, 8) (8, 5410) (5410,)


#### Drop irrelevant topics

In [70]:
W1_clean = W1[:, [0,2,3,6,7]]
H1_clean = H1[[0,2,3,6,7], :]
show_topics(H1_clean, num_top_words = 10, vocab = vocab)

['levél kap biztosítás kötelező casco küld gépjármű nap email autó',
 'díj évforduló éves fizet szerződés hó következő biztosítás negyedéves első',
 'bank csoportos megbízás levon von beszedési levonás terhelés beszedés tud',
 'utal összeg közlemény számlaszám lát ír utalás pénz ügyfél számla',
 'csekk küld befizet fizet kap hó negyedév darab levél tud']

In [67]:
pd.DataFrame(show_topics(H1_clean, num_top_words = 10, vocab = vocab), columns = ['top_words'])

,top_words
0,levél kap biztosítás kötelező casco küld gépjá...
1,díj évforduló éves fizet szerződés hó következ...
2,bank csoportos megbízás levon von beszedési le...
3,utal összeg közlemény számlaszám lát ír utalás...
4,csekk küld befizet fizet kap hó negyedév darab


In [68]:
pd.DataFrame(W1_clean.argmax(axis = 1), columns = ['key'])

,key
0,1
1,1
2,1
3,3
4,1
...,...
995,1
996,1
997,4
998,0


In [69]:
df = pd.DataFrame(W1_clean.argmax(axis = 1), columns = ['topic']).groupby(['topic'])['topic'].count()
df

topic
0    193
1    244
2    153
3    243
4    167
Name: topic, dtype: int64